In [102]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [103]:
#direktori data untuk training dan testing
trainingDataDir = "animal_data"
# testingDataDir = "/kaggle/input/uts-praktikum-artificial-intelligence/testing/testing"

In [104]:
# Banyaknya Epoch
NUM_EPOCHS = 30
class MyCallBackClass(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        global NUM_EPOCHS
        if logs.get("accuracy") > 0.98 and logs.get("val_accuracy") > 0.98:
            print(
                "\nTelah Mencapai Target, Training Berhenti di epoch",
                epoch + 1,
                "akurasi training:",
                logs.get("accuracy"),
                "akurasi valiadasi:",
                logs.get("val_accuracy"),
            )
            self.model.stop_training = True
        elif epoch == NUM_EPOCHS - 1:
            print(
                "\nEpoch",
                NUM_EPOCHS,
                "Model Masih Belum Mencapai Target",
            )

In [105]:
def dataPreprocessing(trainingDataDir):
    # train_df = createDataFrame(trainingDataDir)
    # test_df = createDataFrame(testingDataDir)
    
    # train_datagen = ImageDataGenerator(
    #     rescale=1.0 / 255,
    #     validation_split=0.2,
    # )
    # print (train_df)
    # test_datagen = ImageDataGenerator(rescale=1.0 / 255)
    
    training_datagen = ImageDataGenerator(
        rescale=1/255.,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2  
    )

    # YOUR IMAGE SIZE SHOULD BE 150x150
    # Make sure you used "categorical"
    train_generator = training_datagen.flow_from_directory(
        trainingDataDir,
        target_size=(150,150),
        class_mode='categorical',
        batch_size=32,
        subset='training'  
    )

    validation_generator = training_datagen.flow_from_directory(
        trainingDataDir,
        target_size=(150,150),
        class_mode='categorical',
        batch_size=32,
        subset='validation'  
    )

    #y_col=none, karena data test dugunakan untuk prediksi sehingga tidak perlu label (class)
    # test_generator = test_datagen.flow_from_dataframe(
    #     dataframe=test_df,
    #     directory=testingDataDir,
    #     x_col="FileName",
    #     y_col=None,
    #     target_size=(64, 64),
    #     batch_size=1,
    #     class_mode=None,
    #     shuffle=False,
    # )

    return train_generator, validation_generator#, test_generator

In [106]:
#Membuat generator untuk training, validation, dan testing
train_generator, validation_generator = dataPreprocessing(trainingDataDir)

Found 1561 images belonging to 15 classes.
Found 383 images belonging to 15 classes.


In [107]:
def buildModel():
    
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dropout(0.5),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dense(15, activation='softmax')
    ])
    
    #mengecek informasi parameter dalam model
    model.summary()
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [108]:
#buat model
myModel = buildModel()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,830,799 (18.43 MB)

 Trainable params: 4,830,543 (18.43 MB)

 Non-trainable params: 256 (1.00 KB)

In [109]:
def trainModel(model, train_generator, validation_generator):
    callbacks = MyCallBackClass()

    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=NUM_EPOCHS,
        callbacks=[callbacks],
    )

In [110]:
#melatih model
trainModel(myModel, train_generator, validation_generator)

Epoch 1/30


C:\Users\LENOVO\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


49/49 ━━━━━━━━━━━━━━━━━━━━ 36s 659ms/step - accuracy: 0.1111 - loss: 2.8462 - val_accuracy: 0.1175 - val_loss: 2.6495
Epoch 2/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 20s 390ms/step - accuracy: 0.2046 - loss: 2.4168 - val_accuracy: 0.1488 - val_loss: 3.0904
Epoch 3/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 20s 393ms/step - accuracy: 0.2694 - loss: 2.2914 - val_accuracy: 0.1619 - val_loss: 2.9048
Epoch 4/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 20s 390ms/step - accuracy: 0.3284 - loss: 2.0908 - val_accuracy: 0.3055 - val_loss: 2.2421
Epoch 5/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 20s 388ms/step - accuracy: 0.3810 - loss: 1.9786 - val_accuracy: 0.3159 - val_loss: 2.0906
Epoch 6/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 20s 393ms/step - accuracy: 0.4244 - loss: 1.8255 - val_accuracy: 0.3551 - val_loss: 2.0515
Epoch 7/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 20s 388ms/step - accuracy: 0.4291 - loss: 1.7256 - val_accuracy: 0.3003 - val_loss: 2.2488
Epoch 8/30
49/49 ━━━━━━━━━━━━━━━━━━━━ 22s 422ms/step - accuracy: 0.4652 - loss: 1.6763 - val_accuracy: 0.355